### Sequence Iterator

In [5]:
class SequenceIterator:
    """An iterator for any of the Python sequence type"""
    def __init__(self,sequence):
        """Create an interator for a given sequence"""
        self._seq = sequence # a copy of the given sequence
        self._k = -1  # counter, it will be 0 when next is used

    def __next__(self):
        """Return the next element or else raise StopIteration Error"""
        self._k += 1
        if self._k < len(self._seq):
            return self._seq[self._k]
        else:
            raise StopIteration() # no more elements to iterate

    def __iter__(self):
        """By convention, an interator must return itself"""
        return self

In [9]:
lst = list(range(10))
si = SequenceIterator(lst)

### Making the Range Class

In [48]:
class Range:
    """A class to mimic the actual range class.
    """
    def __init__(self, start, stop = None, step = 1):
        """Instance is same as built-in range."""
        if step == 0:
            raise ValueError("Step cannot be Zero.")

        if stop == None: # special case of Range(n)
            start, stop = 0, start

        self._length = max(0, (stop - start + step -1) // step) #effective length

        # need knowledge of start and step but not of stop to use __getitem__
        self._start, self._step = start, step

    def __len__(self):
        """Returns the number of entries in the range"""
        return self._length

    def __getitem__(self, k):
        """Return entry at index k"""
        if k < 0:
            k += len(self) #converting negative index into positive
        if not 0 <= k < self._length:
            raise IndexError("Index is out of range.")

        return self._start + k * self._step


In [49]:
r = Range(8,140,5)

In [51]:
len(r)

27

### Predatory Credit Card

In [52]:
# We will be using the class CreditCard as the parent class so lets import it
import creditcard

In [55]:
class PredatoryCreditCard(creditcard.CreditCard):
    """A class for predatory credit card which extends the class of credit card to include interests and fees"""

    def __init__(self, customer, bank, acnt, limit, apr):
        """
        Create a predatory credit card instance.
        The initial balance is zero.
        customer:   Name
        bank:       Name of the bank
        acnt:       Account Number
        limit:      Credit limit (is a number)
        apr:        APR (eg. 0.1 for 10%)
        """
        super().__init__(customer, bank, acnt, limit)
        self._apr = apr

    def charge(self, price):
        """
        Charge given price to the credit card, if the limit is sufficient
        Return True, if charge was processed
        Retur False and assess $5 if charge was denied
        """
        success = super().charge(price) #calling inherited method
        if not success:
            self._balance += 5 # adding penalty
        return success # return success to the caller

    def process_month(self):
        """ Assess monthly interests to the outstanding balance"""
        if self._balance > 0:
            # if positive balance, conver apr to multiplicative factor
            monthly_factor = pow(1 + self._apr, 1/12)
            self._balance *= monthly_factor

In [59]:
pcc = PredatoryCreditCard("SK","WF","1234",500,0.08)
pcc.charge(200) # Charge within credit limit
print(pcc.get_balance())
pcc.charge(400) #charge beyond credit limit
print(pcc.get_balance())

200
205


## Progression
### Default Progression

In [66]:
class Progression:
    """Iterator producing a generic progression
    By default it produces whole numbers 0, 1, 2, ...
    """
    def __init__(self, start= 0):
        """Initizlize the give value to the first iterated value"""
        self._current = start

    def _advance(self):
        """
        Update self._current to a new value
        This should be overridden by a subclass to customize progression.
        By convention, if current is set to None, it means the end of a finite progression.
        """
        self._current += 1

    def __next__(self):
        """Return the next element, or else raise the StopIteration Error"""
        if self._current is None: # our convention to end a progression
            raise StopIteration()
        else:
            answer = self._current #current value
            self._advance() #preparing for next step
            return answer #return the current value

    def __iter__(self):
        """By convention, an iterator must return itself"""
        return self

    def print_progression(self,n):
        """Print the next n values of a progression"""
        print(" ".join(str(next(self)) for j in range(n)))

In [75]:
p = Progression()

In [76]:
p.print_progression(10)

0 1 2 3 4 5 6 7 8 9


### Arithmetic Progression

In [77]:
class ArithmeticProgression(Progression):
    """Defines sub-class of Arithmetic progression from the default progression class"""
    def __init__(self, increment = 1, start = 0):
        """Instance for the class with default start = 0 and increment = 1
        increment: step for increment
        start:      starting value
        """
        super().__init__(start) #initiating using the base class
        self._increment = increment

    def _advance(self):
        """
        Advances the progression with the increment
        """
        self._current += self._increment


In [79]:
ap = ArithmeticProgression(5)

In [83]:
next(ap)

15

### Geometric Progreesion

In [88]:
class GeometricProgression(Progression):
    """Defines sub-class of Arithmetic progression from the default progression class"""
    def __init__(self, base = 1, start = 1):
        """Instance for the class with default start = 0 and base = 1
        base: step for base
        start: starting value
        """
        super().__init__(start) #initiating using the base class
        self._base = base

    def _advance(self):
        """
        Advances the progression with the base
        """
        self._current *= self._base

In [95]:
gp = GeometricProgression(5,1)

In [96]:
gp.print_progression(5)

1 5 25 125 625


### Fibonacci Progression

In [97]:
class FibonacciProgression(Progression):
    """Specializes the progression class to Fibonacci Progression"""
    def __init__(self, first = 0, second = 1):
        """Instance of the Fibonacci Progression
        first is the first one in the sequence and 
        second is the second one in the sequence
        """
        super().__init__(first)
        self._second = second

    def _advance(self):
        """Returns the next entry in the Fibonacci Sequence"""
        self._current, self._second = self._second, self._current + self._second

In [99]:
FibonacciProgression(0,1).print_progression(30)

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229


In [101]:
FibonacciProgression(21,34).print_progression(5)

21 34 55 89 144


### Range Using metaclass

In [1]:
import sequence # the module I just wrote where I defined a sequence

In [2]:
class Range(sequence.Sequence): # we will get the definition of index and contains
    """A class to mimic the actual range class.
    """
    def __init__(self, start, stop = None, step = 1):
        """Instance is same as built-in range."""
        if step == 0:
            raise ValueError("Step cannot be Zero.")

        if stop == None: # special case of Range(n)
            start, stop = 0, start

        self._length = max(0, (stop - start + step -1) // step) #effective length

        # need knowledge of start and step but not of stop to use __getitem__
        self._start, self._step = start, step

    def __len__(self):
        """Returns the number of entries in the range"""
        return self._length

    def __getitem__(self, k):
        """Return entry at index k"""
        if k < 0:
            k += len(self) #converting negative index into positive
        if not 0 <= k < self._length:
            raise IndexError("Index is out of range.")

        return self._start + k * self._step


In [3]:
Range(5).count(3)

1

### Shallow Vs Deep Copying

In [4]:
x=4
y=x
print(y)
x=7
print(y)

4
4


In [6]:
x=[1,2,3,4]
y=x
print(y)
x=7
print(y)
print(x)

[1, 2, 3, 4]
[1, 2, 3, 4]
7


In [9]:
y =range(10)
x= y
x=2
print(y)

range(0, 10)


In [13]:
x = [1,2,3]
y = x
x.append(4)
print(y)
x.remove(4) #changing x also changes y, because they are pointing to the same instance of list class
print(y)

[1, 2, 3, 4]
[1, 2, 3]


In [16]:
import copy
x = [1,2,3,4]
y = x
print(y)
z = copy.deepcopy(x)
print(z)
w = copy.copy(x)
x.append(5)
print(x)
print(y)
print(z)
print(w)

[1, 2, 3, 4]
[1, 2, 3, 4]
[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]
[1, 2, 3, 4]
[1, 2, 3, 4]


In [18]:
w = [1,2,3,4]
x = w
y = copy.copy(w)
z= copy.deepcopy(w)
print(w)
print(x)
print(y)
print(z)
w.append(5) # changing w
print(w)
print(x)
print(y)
print(z)
x.remove(1)
print(w)
print(x)
print(y)
print(z)
y.remove(2)
print(w)
print(x)
print(y)
print(z)
z.remove(3)
print(w)
print(x)
print(y)
print(z)


[1, 2, 3, 4]
[1, 2, 3, 4]
[1, 2, 3, 4]
[1, 2, 3, 4]
[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]
[1, 2, 3, 4]
[1, 2, 3, 4]
[2, 3, 4, 5]
[2, 3, 4, 5]
[1, 2, 3, 4]
[1, 2, 3, 4]
[2, 3, 4, 5]
[2, 3, 4, 5]
[1, 3, 4]
[1, 2, 3, 4]
[2, 3, 4, 5]
[2, 3, 4, 5]
[1, 3, 4]
[1, 2, 4]


In [19]:
x = [1,2,3,4]
y = copy.copy(x)
z= copy.deepcopy(x)
x[1] = 7
print(x)
print(y)
print(z)

[1, 7, 3, 4]
[1, 2, 3, 4]
[1, 2, 3, 4]


In [20]:
x = [1,2,3,4]
y = copy.copy(x)
z= copy.deepcopy(x)
y[1] = 7
print(x)
print(y)
print(z)

[1, 2, 3, 4]
[1, 7, 3, 4]
[1, 2, 3, 4]


In [21]:
x = [1,2,3,4]
y = copy.copy(x)
z= copy.deepcopy(x)
z[1] = 7
print(x)
print(y)
print(z)

[1, 2, 3, 4]
[1, 2, 3, 4]
[1, 7, 3, 4]
